# Архитектура RNN. Классификация текста.

Привет! В это семинаре мы познакомимся с задачей классификации текста, на примере поиска тематики новости, а также с двумя из основных архитектурт рекуррентных нейросетей – RNN и GRU.

Нам потребуется одна библиотека от `HuggingFace🤗` под названием `datasets`. Она содержит большое число датасетов, которые используются в NLP.

In [1]:
!pip install datasets

     |████████████████████████████████| 365 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 140 kB 23.6 MB/s eta 0:00:01
     |████████████████████████████████| 24.0 MB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 8.0 MB/s  eta 0:00:01
     |████████████████████████████████| 132 kB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 6.0 MB/s eta 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.8.1
    Uninstalling fsspec-2021.8.1:
      Successfully uninstalled fsspec-2021.8.1


In [2]:
import random
import numpy as np

import nltk
import gensim.downloader as api

import torch
import torch.nn as nn
import datasets

In [3]:
# За детерминизм!
SEED = 0xDEAD
random.seed(SEED)
np.random.seed(SEED)
torch.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)

Загрузим датасет новостей: `AgNews`. В нем разделены тексты на 4 темы: `World`, `Sports`, `Business`, `Sci/Tech`. Посмотрим на структуру датасета и на примеры текстов:

In [4]:
dataset = datasets.load_dataset("ag_news")
dataset["train"]

Using custom data configuration default


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /Users/antonermak/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

In [5]:
dataset["train"][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

В `dataset` находятся `train` и `test` части датасета.

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

Чтобы превращать текст из набора слов в набор векторов мы будем использовать предобученные эмбеддинги. Посмотрим на их список и выберем один из них.

In [7]:
print("\n".join(api.info()['models'].keys()))

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [8]:
word2vec = api.load("glove-twitter-50")

[================================================--] 97.1% 193.8/199.5MB downloaded


Токенезируем наш текст с помощью NLTK.

In [9]:
MAX_LENGTH=128

tokenizer = nltk.WordPunctTokenizer()

dataset = dataset.map(
    lambda item: {
        "tokenized": tokenizer.tokenize(item["text"])[:MAX_LENGTH]
    }
)

  0%|          | 0/120000 [00:00<?, ?ex/s]

  0%|          | 0/7600 [00:00<?, ?ex/s]

Создадим мапинг из токенов в индексы

In [10]:
word2idx = {word: idx for idx, word in enumerate(word2vec.index2word)}

Переведем токены в индексы

In [11]:
def encode(word):
    if word in word2idx.keys():
        return word2idx[word]
    return word2idx["unk"]

In [12]:
dataset = dataset.map(
    lambda item: {
        "features": [encode(word) for word in item["tokenized"]]
    }
)

  0%|          | 0/120000 [00:00<?, ?ex/s]

  0%|          | 0/7600 [00:00<?, ?ex/s]

In [13]:
dataset["train"][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2,
 'tokenized': ['Wall',
  'St',
  '.',
  'Bears',
  'Claw',
  'Back',
  'Into',
  'the',
  'Black',
  '(',
  'Reuters',
  ')',
  'Reuters',
  '-',
  'Short',
  '-',
  'sellers',
  ',',
  'Wall',
  'Street',
  "'",
  's',
  'dwindling',
  '\\',
  'band',
  'of',
  'ultra',
  '-',
  'cynics',
  ',',
  'are',
  'seeing',
  'green',
  'again',
  '.'],
 'features': [62980,
  62980,
  1,
  62980,
  62980,
  62980,
  62980,
  13,
  62980,
  17,
  62980,
  20,
  62980,
  28,
  62980,
  28,
  49286,
  4,
  62980,
  62980,
  48,
  137,
  214902,
  370,
  1645,
  39,
  8606,
  28,
  380053,
  4,
  70,
  1321,
  1745,
  389,
  1]}

In [16]:
dataset.remove_columns(["text", "tokenized"])

DatasetDict({
    train: Dataset({
        features: ['label', 'features'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['label', 'features'],
        num_rows: 7600
    })
})

Переведем в тензоры

In [17]:
dataset.set_format(type='torch')

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'tokenized', 'features'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label', 'tokenized', 'features'],
        num_rows: 7600
    })
})

In [18]:
dataset["train"][0]

TypeError: new(): invalid data type 'numpy.str_'

Хотим склеить объекты разной длинны в батчи. Для этого давайте напишем `collate_fn`.

In [19]:
def collate_fn(batch):
    max_len = max(len(row["features"]) for row in batch)
    input_embeds = torch.empty((len(batch), max_len), dtype=torch.long)
    labels = torch.empty(len(batch), dtype=torch.long)
    for idx, row in enumerate(batch):
        to_pad = max_len - len(row["features"])
        input_embeds[idx] = torch.cat((row["features"], torch.zeros(to_pad)))
        labels[idx] = row["label"]
    return {"features": input_embeds, "labels": labels}

In [ ]:
from torch.utils.data import DataLoader

loaders = {
    k: DataLoader(
        ds, shuffle=(k=="train"), batch_size=32, collate_fn=collate_fn
    ) for k, ds in dataset.items()
}

## CNN

Первая модель, которую мы рассмотрим: CNN. Одномерная конволюция достаточно хорошо справляется с задачей классификации. В конце надо собрать вектор текста с помощью `AdaptiveMaxPool1d` или `AdiptiveAvgPool1d`. Для классиффикации можно собрать любую Feed Forward Network.

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=4):
        super().__init__()
        self.embeddings = nn.Embedding(len(word2idx), embedding_dim=embed_size)
        self.cnn = nn.Sequential(
            nn.Conv1d(embed_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1),
            nn.Flatten(),
        )
        self.cl = nn.Sequential(
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        x = self.embeddings(x)  # (batch_size, seq_len, embed_dim)
        x = x.permute(0, 2, 1)
        x = self.cnn(x)
        prediction = self.cl(x)
        return prediction

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNNModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1

Подготовим функцию для обучения модели:

In [ ]:

from tqdm.notebook import tqdm, trange


def training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm=2):
    for e in trange(num_epochs, leave=False):
        model.train()
        num_iter = 0
        pbar = tqdm(loaders["train"], leave=False)
        for batch in pbar:
            optimizer.zero_grad()
            input_embeds = batch["features"].to(device)
            labels = batch["labels"].to(device)
            prediction = model(input_embeds)
            loss = criterion(prediction, labels)
            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            num_iter += 1
        valid_loss = 0
        valid_acc = 0
        num_iter = 0
        model.eval()
        with torch.no_grad():
            correct = 0
            num_objs = 0
            for batch in loaders["test"]:
                input_embeds = batch["features"].to(device)
                labels = batch["labels"].to(device)
                prediction = model(input_embeds)
                valid_loss += criterion(prediction, labels)
                correct += (labels == prediction.argmax(-1)).float().sum()
                num_objs += len(labels)
                num_iter += 1

        print(f"Valid Loss: {valid_loss / num_iter}, accuracy: {correct/num_objs}")

In [ ]:
training(model, criterion, optimizer, num_epochs, loaders)

## RNN

Вторая модель: RNN. Это рекуррентная сеть, она использует скрытое состояние из прошлой иттерации для создания нового. Это описывается с помощью формул:

$$
h_t = \tanh(W_{ih} x_t + b_{ih} + W_{hh} h_{(t-1)} + b_{hh})
$$

Напишем этот модуль на `Torch`!



In [ ]:
class RNN(nn.Module):
    def __init__(self, embed_size, hidden_size):
        super().__init__()

        self.embed_size = embed_size
        self.hidden_size = hidden_size

        self.w_h = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_h = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_x = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_x = nn.Parameter(torch.rand(1, hidden_size))

    def forward(self, x, hidden=None):
        '''
        x – torch.FloatTensor with the shape (bs, seq_length, emb_size)
        hidden - torch.FloatTensro with the shape (bs, hidden_size)
        return: torch.FloatTensor with the shape (bs, hidden_size)
        '''
        if hidden is None:
            hidden = torch.zeros((x.size(0), self.hidden_size)).to(x.device)
        seq_length = x.size(1)
        for cur_idx in range(seq_length):
            hidden = torch.tanh(
                x[:, cur_idx] @ self.w_x + self.b_x + hidden @ self.w_h + self.b_h
            )
        return hidden

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=4):
        super().__init__()
        self.embeddings = nn.Embedding(len(word2idx), embed_size)
        self.rnn = RNN(embed_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embeddings(x)
        hidden = self.rnn(x)
        output = self.cls(hidden)
        return output

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = RNNModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [ ]:
training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

## GRU

Третья модель: GRU. Она усложненная версия `RNN`. Гланая идея GRU: гейты. Так реализуется "память" модели – она маскирует часть старого скрытого состояния, создавая на этом месте новое. Модель GRU описывается следующим образом:

$$
\begin{array}{ll}
            r_t = \sigma(W_{ir} x_t + b_{ir} + W_{hr} h_{(t-1)} + b_{hr}) \\
            z_t = \sigma(W_{iz} x_t + b_{iz} + W_{hz} h_{(t-1)} + b_{hz}) \\
            n_t = \tanh(W_{in} x_t + b_{in} + r_t * (W_{hn} h_{(t-1)}+ b_{hn})) \\
            h_t = (1 - z_t) * n_t + z_t * h_{(t-1)}
        \end{array}
$$

In [ ]:
class GRU(nn.Module):
    def __init__(self, embed_size, hidden_size):
        super().__init__()

        self.embed_size = embed_size
        self.hidden_size = hidden_size

        self.w_rh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_rh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_rx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_rx = nn.Parameter(torch.rand(1, hidden_size))

        self.w_zh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_zh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_zx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_zx = nn.Parameter(torch.rand(1, hidden_size))

        self.w_nh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_nh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_nx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_nx = nn.Parameter(torch.rand(1, hidden_size))

    def forward(self, x, hidden = None):
        '''
        x – torch.FloatTensor with the shape (bs, seq_length, emb_size)
        hidden - torch.FloatTensro with the shape (bs, hidden_size)
        return: torch.FloatTensor with the shape (bs, hidden_size)
        '''
        if hidden is None:
            hidden = torch.zeros((x.size(0), self.hidden_size)).to(x.device)
        
        for cur_idx in range(x.size(1)):
            r = torch.sigmoid(
                x[:, cur_idx] @ self.w_rx + self.b_rx + hidden @ self.w_rh + self.b_rh
            )
            z = torch.sigmoid(
                x[:, cur_idx] @ self.w_zx + self.b_zx + hidden @ self.w_zh + self.b_zh
            )
            n = torch.tanh(
                x[:, cur_idx] @ self.w_nx + self.b_nx + r * (hidden @ self.w_nh + self.b_nh)
            )
            hidden = (1 - z) * n + z * hidden

        return hidden

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=4):
        super().__init__()
        self.embed = nn.Embedding(len(word2idx), embed_size)
        self.gru = GRU(embed_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.embed(x)
        hidden = self.gru(x)
        output = self.cls(hidden)
        return output

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = GRUModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [ ]:
training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

## GRU + Embeddings

Мы не просто так загрузили эмбэдинги в начале. Давай использовать их вместо случайной инициализации! Для этого надо немного переделать способ подачи данных в модель и добавить в модель модуль `Embedding`. По-экспериментируем на модели `GRU`.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = GRUModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [ ]:
with torch.no_grad():
    for word, idx in word2idx.items():
        if word in word2vec:
            model.embed.weight[idx] = torch.from_numpy(word2vec.get_vector(word))

In [ ]:
training(model, criterion, optimizer, num_epochs, loaders, max_grad_norm)

Попробуем заморозить эмбединги на первых итерациях обучения. Это поможет не сильно портить наши эмбединги на первых итерациях.

In [ ]:
def freeze_embeddings(model, req_grad=False):
    embeddings = model.embed
    for c_p in embeddings.parameters():
        c_p.requires_grad = req_grad

In [ ]:
def training_freeze(model, criterion, optimizer, num_epochs, loaders, max_grad_norm=2, num_freeze_iter=1000):
    freeze_embeddings(model)
    for e in trange(num_epochs, leave=False):
        model.train()
        num_iter = 0
        pbar = tqdm(loaders["train"], leave=False)
        for batch in pbar:
            if num_iter > num_freeze_iter and e < 1:
                freeze_embeddings(model, True)
            optimizer.zero_grad()
            input_embeds = batch["features"].to(device)
            labels = batch["labels"].to(device)
            prediction = model(input_embeds)
            loss = criterion(prediction, labels)
            loss.backward()
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            num_iter += 1
        valid_loss = 0
        valid_acc = 0
        num_iter = 0
        model.eval()
        with torch.no_grad():
            correct = 0
            num_objs = 0
            for batch in loaders["test"]:
                input_embeds = batch["features"].to(device)
                labels = batch["labels"].to(device)
                prediction = model(input_embeds)
                valid_loss += criterion(prediction, labels)
                correct += (labels == prediction.argmax(-1)).float().sum()
                num_objs += len(labels)
                num_iter += 1

        print(f"Valid Loss: {valid_loss / num_iter}, accuracy: {correct/num_objs}")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = GRUModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [ ]:
with torch.no_grad():
    for word, idx in word2idx.items():
        if word in word2vec:
            model.embed.weight[idx] = torch.from_numpy(word2vec.get_vector(word))

In [ ]:
training_freeze(model, criterion, optimizer, num_epochs, loaders, max_grad_norm)